# RDD Manipulations 

In this section, we will present to you some basics manipulations that we can do with RDD - Resilient Distributed Dataset, the basic abstraction in Spark. 

In resume, "it represents an immutable, partitioned collection of elements that can be operated on in parallel. This class contains the basic operations available on all RDDs, such as map, filter, and persist. In addition, PairRDDFunctions contains operations available only on RDDs of key-value pairs, such as groupByKey and join; DoubleRDDFunctions contains operations available only on RDDs of Doubles; and SequenceFileRDDFunctions contains operations available on RDDs that can be saved as SequenceFiles. All operations are automatically available on any RDD of the right type (e.g. RDD[(Int, Int)] through implicit.

Internally, each RDD is characterized by five main properties:

- A list of partitions 
- A function for computing each split 
- A list of dependencies on other RDDs 
- Optionally, a Partitioner for key-value RDDs (e.g. to say that the RDD is hash-partitioned) 
- Optionally, a list of preferred locations to compute each split on (e.g. block locations for an HDFS file)

All of the scheduling and execution in Spark is done based on these methods, allowing each RDD to implement its own way of computing itself. Indeed, users can implement custom RDDs (e.g. for reading data from a new storage system) by overriding these functions. Please refer to the Spark paper for more details on RDD internals."

(Source : https://spark.apache.org/docs/latest/api/java/org/apache/spark/rdd/RDD.html).

When we talk about RDD, we have actions and transformations. Transformations in RDD means we just convert the type of RDD1 to RDD2 without executing or trying to find out what will be the results, otherwise transformations in RDD won't take much time (ex : map, flatMap, etc.). Actions in RDD means we compute really the results (ex : count, collect, take, etc.).

To illustrate some basics actions and transformations in RDD, let's take a Word Counts example!

### We read data from HDFS. By using sc, we have a RDD in the output

In [ ]:
val data = sc.textFile("hdfs://hupi-factory-02-01-01-01/user/j.larre/big data.txt")

/*
Here we used sc.textFile to read a text file in HDFS, if you want to read a json file  

import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder()
  .appName("Spark SQL basic example") 
  .getOrCreate()

// For implicit conversions like converting RDDs to DataFrames
import spark.implicits._

val df = spark.read.json("examples/src/main/resources/example.json") // we have a Dataset here
*/

data: org.apache.spark.rdd.RDD[String] = hdfs://hupi-factory-02-01-01-01/user/j.larre/big data.txt MapPartitionsRDD[5] at textFile at <console>:67


### We count number of lines in data

In [ ]:
data.count()

res5: Long = 14


14

### We can see all of data, but be careful of this action because if data is too big, it can crash the notebook!

In [ ]:
data.collect()

res21: Array[String] = Array("En termes g?n?raux, la science des donn?es est l'extraction de connaissance d'ensembles de donn?es1,2. Elle emploie des techniques et des th?ories tir?es de plusieurs autres ", domaines plus larges des math?matiques, la statistique principalement, la th?orie de l'information et la technologie de l'information, notamment le traitement de signal, des, " mod?les probabilistes, l'apprentissage automatique, l'apprentissage statistique, la programmation informatique, l'ing?nierie de donn?es, la reconnaissance de formes et l'apprentissage,", " la visualisation, l'analytique proph?tique, la mod?lisation d'incertitude, le stockage de donn?es, la compression de donn?es et le calcul ? haute performance. Les m?thodes qui s'adaptent", " aux donn?es de masse sont particu...

entries total 
 
 
 
 
 
 
 
 
 entries total

### We can also do this way...

In [ ]:
data.collect().foreach(println)

En termes g?n?raux, la science des donn?es est l'extraction de connaissance d'ensembles de donn?es1,2. Elle emploie des techniques et des th?ories tir?es de plusieurs autres 
domaines plus larges des math?matiques, la statistique principalement, la th?orie de l'information et la technologie de l'information, notamment le traitement de signal, des
 mod?les probabilistes, l'apprentissage automatique, l'apprentissage statistique, la programmation informatique, l'ing?nierie de donn?es, la reconnaissance de formes et l'apprentissage,
 la visualisation, l'analytique proph?tique, la mod?lisation d'incertitude, le stockage de donn?es, la compression de donn?es et le calcul ? haute performance. Les m?thodes qui s'adaptent
 aux donn?es de masse sont particuli?rement int?ressantes dans la science des donn?es, bien que la discipline ne soit g?n?ralement pas consid?r?e comme limit?e ? ces donn?es.


Les diff?rentes disciplines de la science des donn?es.
La science des donn?es (en anglais data scien

### But it's always better to take some n lines of data instead of collecting all data

In [ ]:
data.take(2)

res25: Array[String] = Array("En termes g?n?raux, la science des donn?es est l'extraction de connaissance d'ensembles de donn?es1,2. Elle emploie des techniques et des th?ories tir?es de plusieurs autres ", domaines plus larges des math?matiques, la statistique principalement, la th?orie de l'information et la technologie de l'information, notamment le traitement de signal, des)


entries total 
 
 
 
 
 
 
 
 
 entries total

### Next, we need to remove all the special characters in data by using regex. 

To understand more about regex, you can find in this link (http://docs.oracle.com/javase/7/docs/api/java/util/regex/Pattern.html). In this transformation, we use map() to apply the function replaceAll for each element inside RDD.

In [ ]:
val data_without_special_characters = data.map(l => l.replaceAll("""[\p{Punct}]""", ""))

data_without_special_characters: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[7] at map at <console>:69


In [ ]:
data_without_special_characters.take(10)

res29: Array[String] = Array("En termes g?n?raux la science des donn?es est lextraction de connaissance densembles de donn?es12 Elle emploie des techniques et des th?ories tir?es de plusieurs autres ", domaines plus larges des math?matiques la statistique principalement la th?orie de linformation et la technologie de linformation notamment le traitement de signal des, " mod?les probabilistes lapprentissage automatique lapprentissage statistique la programmation informatique ling?nierie de donn?es la reconnaissance de formes et lapprentissage", " la visualisation lanalytique proph?tique la mod?lisation dincertitude le stockage de donn?es la compression de donn?es et le calcul ? haute performance Les m?thodes qui sadaptent", " aux donn?es de masse sont particuli?rement int?ressantes dans ...

entries total 
 
 
 
 
 
 
 
 
 entries total

### Then, we split data into words and convert all into lower case. 

Here we use flatMap (simply equal to map() + "flatten" ). Here we want to transform an RDD of Array of String into an RDD of String and apply the function inside flatMap for each String. 

In [ ]:
val words = data_without_special_characters.flatMap(l => l.split(" ").map(l => l.toLowerCase))

words: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[9] at flatMap at <console>:71


In [ ]:
words.take(10)

<console>:69: error: not found: value words
       words.take(10)
       ^


### We can now count by key to compute the number of occurences of each word

To do this, we have 2 ways to do :

#### 1/ First option :

For each element in RDD, we add 1 column with value 1, then by key (word), we compute the sum of value by using reduceByKey. When we do "_ + _" means everytime we have same key (word), we do the sum of its values.

In [ ]:
val wordCount = words.map(l => (l, 1)).reduceByKey(_ + _)

wordCount: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[11] at reduceByKey at <console>:73


#### 2/ Second option :

We do the same, for each element in RDD, we map with value 1. Then we groupByKey and compute the sum of list of the value

In [ ]:
val wordCount = words.map(l => (l, 1)).groupByKey().map(l => (l._1, l._2.toList.sum))

<console>:67: error: not found: value words
       val wordCount = words.map(l => (l, 1)).groupByKey().map(l => (l._1, l._2.toList.sum))
                       ^


### For example if we want to find the count of "is", we can also do filter

In [ ]:
wordCount.filter(l => l._1 == "is").take(10)

res34: Array[(String, Int)] = Array()


empty array

### So without any filter, the output will be 

In [ ]:
wordCount.take(10)

<console>:69: error: not found: value wordCount
       wordCount.take(10)
       ^


### From RDD to DataFrame and Dataset

We can convert RDD to Dataset or DataFrame. In fact, in Spark, these 3 APIs (RDD, DataFrame ans Dataset) are the three types officially used. They can convert to each other. 

To understand thoroughly when to use them, you can read at https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

#### From RDD to DataFrame

In [ ]:
wordCount

res28: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[18] at map at <console>:73


MapPartitionsRDD[18] at map at <console>:73

In [ ]:
val df = wordCount.toDF("word", "count")

df: org.apache.spark.sql.DataFrame = [word: string, count: int]


In [ ]:
//val wc = df.select("count").sum
val count = df.groupBy().sum().show()

+----------+
|sum(count)|
+----------+
|       198|
+----------+

count: Unit = ()


In [ ]:
al count = df.sum(

In [ ]:
df.show()

+--------------------+-----+
|                word|count|
+--------------------+-----+
|             reading|    1|
|                this|    1|
|               paper|    1|
|  doublerddfunctions|    1|
|          collection|    1|
|          internally|    1|
|                 its|    1|
|                  is|    3|
|       sequencefiles|    1|
|             persist|    1|
|                only|    2|
|                data|    1|
|         abstraction|    1|
|           internals|    1|
|               basic|    2|
|                 map|    1|
|sequencefilerddfu...|    1|
|               class|    1|
|             methods|    1|
|          scheduling|    1|
+--------------------+-----+
only showing top 20 rows



#### From RDD to Dataset

In [ ]:
import org.apache.spark.sql.types.{DoubleType, StringType, StructField, StructType}
import org.apache.spark.sql.{Row, SparkSession}

import org.apache.spark.sql.types.{DoubleType, StringType, StructField, StructType}
import org.apache.spark.sql.{Row, SparkSession}


In [ ]:
val sparkSession = SparkSession.builder
      .master("local")
      .appName("example")
      .getOrCreate()

//val sparkContext = sparkSession.sparkContext
import sparkSession.implicits._

sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@62682cdb
import sparkSession.implicits._


In [ ]:
val dataset = wordCount.toDS()

dataset: org.apache.spark.sql.Dataset[(String, Int)] = [_1: string, _2: int]


In [ ]:
dataset.show()

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|             reading|  1|
|                this|  1|
|               paper|  1|
|  doublerddfunctions|  1|
|          collection|  1|
|          internally|  1|
|                 its|  1|
|                  is|  3|
|       sequencefiles|  1|
|             persist|  1|
|                only|  2|
|                data|  1|
|         abstraction|  1|
|           internals|  1|
|               basic|  2|
|                 map|  1|
|sequencefilerddfu...|  1|
|               class|  1|
|             methods|  1|
|          scheduling|  1|
+--------------------+---+
only showing top 20 rows



In [ ]:
dataset.map(a => a._2).reduce((i,j) => i+j)

res46: Int = 198


198